# MA556 Group Project

## Part 1: Attain Survey Weights

In [20]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.special import expit # Logistic function

# Generate values of z from the normal distribution model
def generate_z(mu_z, sigma_z, N):
    z = np.random.normal(mu_z, sigma_z, N)
    return z

# Calculate the propensity score using logistic regression
def compute_propensity_scores(data, psi, predictors=['Intercept', 'Age', 'Sex', 'Z']):
    # Calculate v'ψ
    linear_predictor = np.dot(data[predictors], psi)

    # Apply the logistic to propensity
    propensity_scores = expit(linear_predictor)

    return propensity_scores

def compute_survey_weights(data, propensity_scores):
    N = len(data)  # Total # individuals
    f = 1 / N  # Sample fraction

    # Calculate W_i
    original_weights = N / propensity_scores

    # Calculate adjusted w_i
    adjusted_weights = f * original_weights / original_weights.mean()

    # Calculate effective sample size n_hat
    n_hat = (original_weights.sum())**2 / (original_weights**2).sum()

    # Calculate final adjusted survey weights
    final_adjusted_weights = n_hat * original_weights / original_weights.sum()

    return final_adjusted_weights


# Import data
data = pd.read_csv('Updated_Project_chromosome.csv')

# Generate values of z
mu_z = 0
sigma_z = 1

N = len(data)
data['Z'] = generate_z(mu_z, sigma_z, N)

# Fit the logistic regression
predictors = ['Intercept', 'Age', 'Sex', 'Z']
response = 'Treatment'

model = sm.Logit(data[response], data[predictors])
results = model.fit()

# The fitted coefficients 'psi'
psi = results.params

# Fit logistic regression
X = data[predictors]
y = data[response]

propensity_scores = compute_propensity_scores(data, psi)
adjusted_weights = compute_survey_weights(data, propensity_scores)

data['AdjustedWeight'] = adjusted_weights

print(data['AdjustedWeight'])


Optimization terminated successfully.
         Current function value: 0.527029
         Iterations 6
0      1.587478
1      1.432752
2      1.391193
3      1.518283
4      0.882995
         ...   
434    0.889431
435    1.276626
436    1.226029
437    1.285220
438    1.270359
Name: AdjustedWeight, Length: 439, dtype: float64


In [15]:
w = data['AdjustedWeight'].to_numpy()

## Depricated Code

In [ ]:
# df = pd.read_csv('data.csv')
# df['Intercept'] = 1
# df['Treatment'] = df['Subject'].apply(lambda x: 0 if 'C' in x else 1)
# df = pd.get_dummies(df, columns=['Sex'], drop_first=True)
# df = df.rename(columns={'Sex_M':'Sex'})

# df.to_csv('data_table.csv')


## Part 2: MCMC

In [24]:
import pymc as pm
import numpy as np

n_samples = len(data.iloc[:39])
x_data = data[['Intercept', 'Treatment', 'Age', 'Sex', 'Z']].iloc[:39].values
y_data = data['y'].iloc[:39].values
treatment_data = data['Treatment'].iloc[:39].values
w_data = data['AdjustedWeight'].iloc[:39].values

with pm.Model() as hierarchical_model:
    #delta_squared = pm.InverseGamma('delta_squared', alpha=0.001, beta=0.001)
    #gamma = pm.Normal('gamma', mu=0, sigma=pm.math.sqrt(delta_squared), shape=5)
    #z_i_mean = pm.math.dot(x_data, gamma)
    #z_i = pm.Normal('z_i', mu=z_i_mean, sigma=pm.math.sqrt(delta_squared / w_data), shape=n_samples)

    # Separate priors for beta0 and beta1
    beta0 = pm.Normal('beta0', mu=0, sigma=1, shape=5)
    beta1 = pm.Normal('beta1', mu=0, sigma=1, shape=5)

    # Standard deviations and correlation for y0i and y1i
    sigma0 = pm.HalfNormal('sigma0', sigma=1)
    sigma1 = pm.HalfNormal('sigma1', sigma=1)
    rho = pm.Uniform('rho', lower=0, upper=1)

    # Bivariate normal for potential outcomes y0i and y1i
    cov = pm.math.stack([[sigma0**2, rho * sigma0 * sigma1],
                         [rho * sigma0 * sigma1, sigma1**2]])
    y0i_mean = pm.math.dot(x_data, beta0)
    y1i_mean = pm.math.dot(x_data, beta1)
    outcomes = pm.MvNormal(
        'outcomes',
        mu=pm.math.stack([y0i_mean, y1i_mean], axis=1),
        cov=cov,
        observed=[[y_data[i],0] if treatment_data[i] == 0 else [0,y_data[i]] for i in range(39)] #need to change this...
    )
    trace = pm.sample(1000, return_inferencedata=True, target_accept=0.95, tune=1500)

## Part 3: Beyesian Predictive Inference

In [48]:
b0 = np.mean(trace.posterior['beta0'], axis= 1)[0]

In [49]:
b1 = np.mean(trace.posterior['beta1'], axis= 1)[0]

In [50]:
print(b0,b1)

<xarray.DataArray 'beta0' (beta0_dim_0: 5)>
array([ 0.55236552, -0.93863932,  0.00532862,  0.13846182, -0.17726761])
Coordinates:
    chain        int64 0
  * beta0_dim_0  (beta0_dim_0) int64 0 1 2 3 4 <xarray.DataArray 'beta1' (beta1_dim_0: 5)>
array([0.12532367, 2.12867659, 0.00259907, 0.07497326, 0.5528719 ])
Coordinates:
    chain        int64 0
  * beta1_dim_0  (beta1_dim_0) int64 0 1 2 3 4
